# OpenStreetMap data wrangling with MongoDB

###### by Alicia Dale; Project 3 of Udacity's Data Analyst Nanodegree Program


What is OSM? Open Street Map is a type of software that is considered to be an example of volunteered geographical information. It currently has over 2 million registered users who contriubte to the Open Street Mapping foundation. With that many users the platform is prone to have many errors with the data that is being added to the software that so many people use. My task here is to take the data from a chosen area and clean it up! Make sure that words are not over abbreviated and that the data is clean and easy to read and utilize to be used in various applications such as FourSquare. Below is the step-by-step process for my data auditing and cleaning.
How do I obtain the data necessary to audit? For this project I had to pick an area that I would like to do my data cleaning on. I chose Philadelphia since it's a place I had never been to and was hoping to learn a bit more about its location through this project. I had done that though this link https://www.openstreetmap.org/relation/188022

### References used for the project: 
###### MongoDB docs: https://docs.mongodb.com/manual/
###### GitHub project examples: https://github.com/dwmercier/Project-3-Data-Analyst-Nanodegree/blob/master/P3%20-%20Data%20Wrangling%20with%20MongoDB.pdf

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  
''' Use cElementTree or lxml if too slow'''
import pprint 
'''pretty printer- produces pleasing representations of your data structures'''
from collections import defaultdict 

OSM_FILE = "/Users/aliciadale/Desktop/philadelphia_pennsylvania.osm"  # This line of code finds my OSM file for Philly
SAMPLE_FILE = "sample.osm" # I am writing to to this file 

k = 10 
''' Parameter: take every k-th top level element'''

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

'''
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

'''

'\nwith open(SAMPLE_FILE, \'wb\') as output:\n    output.write(\'<?xml version="1.0" encoding="UTF-8"?>\n\')\n    output.write(\'<osm>\n  \')\n\n    # Write every kth top level element\n    for i, element in enumerate(get_element(OSM_FILE)):\n        if i % k == 0:\n            output.write(ET.tostring(element, encoding=\'utf-8\'))\n\n    output.write(\'</osm>\')\n\n'

## ITERATIVE PARSING :

##### Since I have the data downloaded successfully I will now begin my auditing and cleaning of the data. I will start with iterative parsing which finds all the top level tags in the document such as, bounds and nodes then I will create a dictionary and add all the tag values to it and count how many of each tag is present in the datafile. Using a sample size file under 2GB of data

In [3]:
"""Parse through the OSM file with ElementTree module to count the number of unique element types"""
def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
                """If element tag is an already found element then count it to the tag name, 
                    if not then create a new tag with the else statement"""
            else:
                tags[elem.tag] = 1
        return tags    

In [4]:
count_tags(OSM_FILE)

{'bounds': 1,
 'member': 55682,
 'nd': 3595591,
 'node': 2959713,
 'osm': 1,
 'relation': 4601,
 'tag': 1803393,
 'way': 291519}

##### The output illustrated above is a dictionary of all the different types of tags followed by their count or how often they appear in the philadephia_pennsylvania.osm file.

## TAG TYPES :

##### In further exploration of my OSM data file, I would like to check the "k" value for each tag element in the data, and find all potential problems. I created 3 expresions to check for certain paterns in the OSM file tags. I would like to change the original format of 
"addr:street"
to 
{"address":{ "street" : "some value"}}

(In order to change the format, I would like to see if we have any tags with problematic characters.)

In [7]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":

        if lower.search(element.attrib['k']):
            keys['lower'] = keys['lower'] + 1
        

        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] = keys['lower_colon'] + 1

        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] = keys['problemchars'] + 1
   
        else:
            keys['other'] = keys['other'] + 1 
        pass
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


keys = process_map(OSM_FILE)
pprint.pprint(keys)

{'lower': 961953, 'lower_colon': 660978, 'other': 180457, 'problemchars': 5}


##### The output presented above showcases all of the tags in my OSM file that fall into the 4 categories that I have created and gives me a count of problem characters in the file. That output is 5. So 5 tags have problematic characters that I will need to evaluate further in my analysis.

## EXPLORING USERS:

##### I want to find how many unique users have contriubted to OSM for the city of Philadelphia. I will need to write a function that returns a set of unique user ID's extracted from my OSM file

In [10]:
"""people invovlved in the map editing """
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for i in element:
            if 'uid' in i.attrib:
                users.add(i.attrib['uid'])
    return users
users = process_map(OSM_FILE)
len(users)

1967

##### The above output shows that 1967 users have contributed to Philadelphia's OSM data

## Auditing Street Names:

##### Here I would like to find all potential errors that I have found in the document that deal with expected common address errors. Such as the word "road" being abbreviated as rd as an example. After careful review of the document I found these problematic address errors.

In [13]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Avenue", "Boulevard", "Court", "Circle", "Drive", "Exit", "Highway", "Lane",
            "Parkway", "Pike", "Place", "Road", "Square", "Suite", "Street", "Trail", "Terrace"]

"""the above "expected" variable contains all of the full words that can be contained in an address in my file
the below "mapping" variable contains all of the mistake abbreviations that have been used in my OSM file and 
shows their proper format"""


mapping = { "Av" : "Avenue",
            "avenue" : "Avenue",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "ave" : "Avenue",
            "Blvd" : "Boulevard",
            "Crt" : "Court",
            "Ct" : "Court",
            "Cir" : "Circle",
            "Dr" : "Drive",
            "drive" : "Drive",
            "Ext" : "Exit",
            "ext" : "Exit",
            "lane" : "Lane",
            "Ln" : "Lane",
            "PIke" : "Pike",
            "Rd" : "Road",
            "Rd." : "Road",
            "rd" : "Road",
            "road" : "Road",
            "Hwy" : "Highway",
            "Sreet" : "Street",
            "st" : "Street", 
            "ST" : "Street",
            "St": "Street",
            "St.": "Street",
            "Atreet" : "Street",
            "Sstreet" : "Street",
            "street" : "Street",
            "Steet" : "Street",
            "Sq" : "Square",
            "Ste" : "Suite",
           "Trl" : "Trail",
            "Ter" : "Terrace"}


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()  
        """groups street types together"""
        if street_type not in expected:
            street_types[street_type].add(street_name)
            """if street type is problematic add it to audit_street_type"""


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street") 
"""Searches K tags with the specific "addr:street" attribute"""


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

philly_street_types = audit(OSM_FILE)
pprint.pprint(dict(philly_street_types)) #pprints dictionary created of all street types from OSM file

{'1': set(['Easton Rd #1',
           'Route 1',
           'S Newtown Street Rd #1',
           'Walnut St #1']),
 '111': set(['South Clinton Avenue Ste. 111']),
 '168': set(['Marlton Pike East Ste. 168']),
 '19047': set(['200 Manor Ave. Langhorne, PA 19047',
               '2245 E. Lincoln Hwy, Langhorne, PA 19047',
               '2275 E Lincoln Hwy, Langhorne, PA 19047',
               '2300  East Lincoln Highway, Pennsylvania 19047']),
 '19067': set(['East Trenton Avenue Morrisville, PA 19067']),
 '205': set(['Office Center Dr #205']),
 '206': set(['US 206', 'US 70 & US 206']),
 '315': set(['Heritage Center Dr #315']),
 '33': set(['33', 'Route 33']),
 '37th': set(['N 37th']),
 '38': set(['New Jersey 38', 'Route 38', 'State Route 38']),
 '39th': set(['N 39th']),
 '40': set(['1140 US Highway 40', 'Rt 40']),
 '4080': set(['4080']),
 '41st': set(['S. 41st']),
 '43rd': set(['N 43rd']),
 '446-1234': set(['1 Brookline BlvdHavertown, PA 19083(610) 446-1234']),
 '452': set(['Market Street;

##### Now we will update the name of the streets. We will take the old name and update it to the new street name that is desired

In [15]:
def update_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)
    return name

for street_type, ways in philly_street_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping, street_type_re)
        print name, "=>", better_name # update street names using better name 
           

1 Brookline BlvdHavertown, PA 19083(610) 446-1234 => 1 Brookline BlvdHavertown, PA 19083(610) 446-1234
Pennel Road; Pennsylvania Route 452 => Pennel Road; Pennsylvania Route 452
Market Street; Pennsylvania Route 452 => Market Street; Pennsylvania Route 452
52 Centerton Rd => 52 Centerton Road
Clements Bridge Rd => Clements Bridge Road
58 Centerton Rd => 58 Centerton Road
E County Line Rd => E County Line Road
Darby Rd => Darby Road
Stokes Rd => Stokes Road
Church Rd => Church Road
York Rd => York Road
Valley Rd => Valley Road
62 Centerton Rd => 62 Centerton Road
South Easton Rd => South Easton Road
Durham Rd => Durham Road
Lincoln Mill Rd => Lincoln Mill Road
Evesham Rd => Evesham Road
Barren Hill Rd => Barren Hill Road
Bristol Rd => Bristol Road
24 Centerton Rd => 24 Centerton Road
Edison Furlong Rd => Edison Furlong Road
Easton Rd => Easton Road
Hulmeville Rd => Hulmeville Road
70 Centerton Rd => 70 Centerton Road
Grove Rd => Grove Road
50 Centerton Rd => 50 Centerton Road
22 Centert

##### I reused some code snippets from the street abbrevation function earlier in this project to check for correct uniform zip codes in the data file. Below is my code for auditing zip codes

In [17]:
from collections import defaultdict # add imports to python code 

def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
    
    elif twoDigits != 95:
        invalid_zipcodes[twoDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode") 
"""look for k attribute tags with addr:postcode format to audit""" 

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)): # start auditing 

        if elem.tag == "node" or elem.tag == "way": # allocating correct areas for audit 
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

philly_zipcode = audit_zip(OSM_FILE) # run audit_zip funtion against OSM_FILE 
pprint.pprint(dict(philly_zipcode)) ##pprint output from zipcodes in file 

{'08': set(['08002',
            '08003',
            '08009',
            '08010',
            '08012',
            '08021',
            '08026',
            '08028',
            '08030',
            '08031',
            '08033',
            '08033-2001',
            '08034',
            '08035',
            '08037',
            '08043',
            '08046',
            '08048',
            '08051',
            '08052',
            '08053',
            '08054',
            '08055',
            '08057',
            '08060',
            '08061',
            '08062',
            '08062-4446',
            '08063',
            '08065',
            '08066',
            '08068',
            '08069',
            '08070',
            '08071',
            '08077',
            '08080',
            '08081',
            '08084',
            '08085',
            '08086',
            '08088',
            '08091',
            '08094',
            '08096',
            '08097',
            '08098',
   

In [18]:
"""The code block below returns all updated zip-codes from the OSM file"""

'The code block below returns all updated zip-codes from the OSM file'

In [19]:
import re #imports reg expressions into python
def update_zip(user_entry):
    """create update_zip function to begin updating zip-codes in file"""
    zipcodes_in_entry = re.findall(r'(\d{5}(-\d{4})?)', user_entry)

    if len(zipcodes_in_entry) > 0: 
        return zipcodes_in_entry[0][0]
    else:
        return 'Invalid'

for street_type, ways in philly_zipcode.iteritems():
    for name in ways:
        better_name = update_zip(name)
        print name, "=>", better_name

NJ 08033 => 08033
NJ 08083 => 08083
NJ 08107 => 08107
1713 => Invalid
1719 => Invalid
1723 => Invalid
19810 => 19810
19014 => 19014
19015 => 19015
19016 => 19016
19010 => 19010
19012 => 19012
19013 => 19013
19018 => 19018
19317 => 19317
19803-9997 => 19803-9997
19314 => 19314
19312 => 19312
19087-3696 => 19087-3696
19406 => 19406
19405 => 19405
19151 => 19151
19150 => 19150
19401 => 19401
19152 => 19152
19468 => 19468
19464 => 19464
19460 => 19460
19462 => 19462
19061 => 19061
19060 => 19060
19063 => 19063
19064 => 19064
19067 => 19067
19066 => 19066
19380 => 19380
19382 => 19382
19073-3299 => 19073-3299
19301 => 19301
19010-3224 => 19010-3224
19148 => 19148
19149 => 19149
19146 => 19146
19147 => 19147
19144 => 19144
19145 => 19145
19147‎ => 19147
19143 => 19143
19140 => 19140
19141 => 19141
19478 => 19478
19477 => 19477
19474 => 19474
19473 => 19473
19072 => 19072
19073 => 19073
19070 => 19070
19076 => 19076
19074 => 19074
19075 => 19075
19078 => 19078
19079 => 19079
19703 => 19703
19

In [20]:
"""Preparing the data set for MongoDB manipulation by converting XML  to JSON"""

'Preparing the data set for MongoDB manipulation by converting XML  to JSON'

In [21]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
from collections import defaultdict



lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
postal_codes = re.compile(r'^[ABCEGHJKLMNPRSTVXY][0-9][ABCEGHJKLMNPRSTVWXYZ][\s]?[0-9][ABCEGHJKLMNPRSTVWXYZ][0-9]')
street_types = re.compile(r'\b\S+\.?$', re.IGNORECASE)


OSM_FILE = "/Users/aliciadale/Desktop/philadelphia_pennsylvania.osm"

CREATED = ["version", "changeset", "timestamp", "user", "uid"]


expected = ["Avenue", "Boulevard", "Court", "Circle", "Drive", "Exit", "Highway", "Lane",
            "Parkway", "Pike", "Place", "Road", "Square", "Suite", "Street", "Trail", "Terrace"]

'''the above "expected" variable contains all of the full words that can be contained in an address in my file
the below "mapping" variable contains all of the mistake abbreviations that have been used in my OSM file and 
shows their proper format'''


street_mapping = { "Av" : "Avenue",
            "avenue" : "Avenue",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "ave" : "Avenue",
            "Blvd" : "Boulevard",
            "Crt" : "Court",
            "Ct" : "Court",
            "Cir" : "Circle",
            "Dr" : "Drive",
            "drive" : "Drive",
            "Ext" : "Exit",
            "ext" : "Exit",
            "lane" : "Lane",
            "Ln" : "Lane",
            "PIke" : "Pike",
            "Rd" : "Road",
            "Rd." : "Road",
            "rd" : "Road",
            "road" : "Road",
            "Hwy" : "Highway",
            "Sreet" : "Street",
            "st" : "Street", 
            "ST" : "Street",
            "St": "Street",
            "St.": "Street",
            "Atreet" : "Street",
            "Sstreet" : "Street",
            "street" : "Street",
            "Steet" : "Street",
            "Sq" : "Square",
            "Ste" : "Suite",
           "Trl" : "Trail",
            "Ter" : "Terrace"}

def shape_element(element):
    """Parses element from iterparse, cleans address, and returns dictionary"""
    node = {}
    if element.tag == "way":
    	  node['node_refs'] = []
    if element.tag == "node" or element.tag == "way":
        node['type'] = element.tag
        attrs = element.attrib 
        node['created'] = {}
        for attr in attrs: 
            if attr == "lat" or attr == "lon":
                if 'pos' not in node:
                    node['pos'] = []
                if attr == 'lat':
                    node['pos'].insert(0,float(element.attrib[attr]))
                elif attr == 'lon':
                    node['pos'].insert(1,float(element.attrib[attr]))
            elif attr in CREATED:
        	    node['created'][attr] = element.attrib[attr]
            else:
                node[attr] = element.attrib[attr]    
        for subtag in element.iter('tag'):
            key, value = subtag.attrib['k'], subtag.attrib['v']
            if problemchars.match(key):
                continue
            elif lower_colon.match(key):
                subtagjsonkey = key.split(':')
                if subtagjsonkey[0] == 'addr':
                    if 'address' not in node:
                    	  node['address'] = {}
                    if subtagjsonkey[1] == 'street':
                        node['address'][subtagjsonkey[1]] = update_name(value, mapping)
                    elif subtagjsonkey[1] == 'postcode':
                        node['address'][subtagjsonkey[1]] = update_postcode(value)
                elif subtagjsonkey[0] == 'turn':
                    continue
                else:
                    node[subtagjsonkey[1]] = value             
            else:
                if ':' not in key:
                    if key == "exit_to":
                        node[key] = update_name(value, mapping)
                    else:
                        node[key] = value
        if element.tag == "way":                    
            for subtag in element.iter('nd'):
                node['node_refs'].append(subtag.attrib['ref'])      
        return node
    else:
        return None



def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

    data = process_map('OSM_FILE', True)  

##### Add import for MongoDB by using pymongo, and connecting to MongoClient

In [45]:
import signal
import os
import subprocess
pro = subprocess.Popen('mongod', preexec_fn = os.setsid)

In [46]:
from pymongo import MongoClient

db_name = 'openstreetmap'

# Connect to Mongo DB
client = MongoClient('localhost:27017')
db = client[db_name]


In [47]:
"""Build mongoimport command instead of using homebrew"""
collection = OSM_FILE[:OSM_FILE.find('.')]
json_file =  OSM_FILE + '.json'

mongoimport_cmd = 'mongoimport -h 127.0.0.1:27017 ' + \
                  '--db ' + db_name + \
                  ' --collection ' + collection + \
                  ' --file ' + json_file

"""Before importing, drop collection if it is already running"""
if collection in db.collection_names():
    print 'Dropping collection: ' + collection
    db[collection].drop()
    
"""Execute the command"""
print 'Executing: ' + mongoimport_cmd
subprocess.call(mongoimport_cmd.split())

Executing: mongoimport -h 127.0.0.1:27017 --db openstreetmap --collection /Users/aliciadale/Desktop/philadelphia_pennsylvania --file /Users/aliciadale/Desktop/philadelphia_pennsylvania.osm.json


0

In [48]:
philadelphia_pennsylvania = db[collection]

##### File Sizes : I want to showcase that my file has successfully converted from OSM to JSON

##### Below I printed out the file size to compare JSON and the original file. The compressed file size that I used for submission was 45.6 MB. The uncompressed files are listed below 

In [68]:
import os
print 'The original OSM file is {} MB'.format(os.path.getsize(OSM_FILE)/1.0e6) #formats OSM file from bytes to megabytes
print 'The JSON file is {} MB'.format(os.path.getsize(OSM_FILE + ".json")/1.0e6)#formats JSON file from bytes to megabytes

The original OSM file is 658.378436 MB
The JSON file is 741.287115 MB


##### Number of Unique Users : Showcasing how many users tells us how many individuals contributed to the Philadelphia open street map dataset

In [69]:
number_of_unique_users = len(philadelphia_pennsylvania.distinct('created.user'))
print 'Number of unique users: {}'.format(number_of_unique_users)

Number of unique users: 1952


##### The JSON file has 10 less users in comparison to the orginal OSM XML file

###### Number of Nodes and Ways 

In [70]:
"""Parse through JSON file to count number of nodes and ways tags"""
print "Number of nodes:",philadelphia_pennsylvania.find({'type':'node'}).count()
print "Number of ways:",philadelphia_pennsylvania.find({'type':'way'}).count()

Number of nodes: 2959699
Number of ways: 290523


###### Number of Unique Users  

In [71]:
len(philadelphia_pennsylvania.distinct('created.user'))#using len to give count, and dot notation to find how many unique user in the dataset

1952

###### Name of top 5 contributers 

In [72]:
"""Find top contributers to the Philadelphia data set"""
result = philadelphia_pennsylvania.aggregate( [
                                         { "$group" : {"_id" : "$created.user", #using dot notation to group id's, with using "created.user" to..
                                        "count" : { "$sum" : 1} } },# count how many users 
                                        { "$sort" : {"count" : -1} }, # sort the count in desc from greatest to least to give us top 5
                                        { "$limit" : 5 } ] ) # limit 5 in the ouputs 

print(list(result))# prints top 5 contriubters 

[{u'count': 797931, u'_id': u'dchiles'}, {u'count': 561902, u'_id': u'woodpeck_fixbot'}, {u'count': 295993, u'_id': u'NJDataUploads'}, {u'count': 106141, u'_id': u'kylegiusti'}, {u'count': 104633, u'_id': u'WesWeaver'}]


### Further data exploration *****

###### Top 10 amenities in Philadelphia 

In [73]:
"""The following aggregation pipeline showcases the top 10 amenities in Philadelphia"""
amenity = philadelphia_pennsylvania.aggregate([{'$match': {'amenity': {'$exists': 1}}}, # finding amenities in the data
                                {'$group': {'_id': '$amenity', 
                                            'count': {'$sum': 1}}}, ## grouping and adding amenities 
                                {'$sort': {'count': -1}}, ## sorting in descending order 
                                {'$limit': 10}])## limits output to top 10 amenities
print(list(amenity))# print list 

[{u'count': 5331, u'_id': u'parking'}, {u'count': 2143, u'_id': u'school'}, {u'count': 1281, u'_id': u'restaurant'}, {u'count': 1125, u'_id': u'place_of_worship'}, {u'count': 523, u'_id': u'fast_food'}, {u'count': 516, u'_id': u'fire_station'}, {u'count': 390, u'_id': u'bank'}, {u'count': 350, u'_id': u'fuel'}, {u'count': 328, u'_id': u'social_facility'}, {u'count': 273, u'_id': u'cafe'}]


###### Top 5 cuisine options 

In [74]:
cuisine = philadelphia_pennsylvania.aggregate([{"$match":{"amenity":{"$exists":1},#finds amenities 
                                 "amenity":"restaurant",}},      
                      {"$group":{"_id":{"Food":"$cuisine"},# groups my cuisine values in the dataset 
                                 "count":{"$sum":1}}}, # counts cuisine values 
                      {"$project":{"_id":0,
                                  "Food":"$_id.Food",
                                  "Count":"$count"}},
                      {"$sort":{"Count":-1}}, 
                      {"$limit":5}])


print(list(cuisine)) # prints output to notebook 

[{u'Food': None, u'Count': 594}, {u'Food': u'pizza', u'Count': 138}, {u'Food': u'italian', u'Count': 68}, {u'Food': u'chinese', u'Count': 66}, {u'Food': u'american', u'Count': 60}]


##### The output showcases that the first count with the most restaurants aren't even CATEGORIZED as a group. So there is potential here to improve the OSM site and to only allow inputs of restaurants with a category of what type of food is served

##### Below I have created an aggregation that finds all places containing the word 'cheesesteak', a famous sandwich that Philadelpia is know for. So just for fun I wanted to see how many places in the data set contained that keyword. Also for fun and for even further analysis I think it would be cool to see if these sandwiches are a college students favorite and if the Philly cheesesteak shops located near campuses are more profitable.

In [75]:
cheesesteak = philadelphia_pennsylvania.aggregate([{"$match":{"amenity":{"$exists":1},#finds amenities 
                                 "amenity":"restaurant",}},      
                      {"$group":{"_id":{"Food":"$cheesesteak"},# groups my philly values in the dataset 
                                 "count":{"$sum":1}}}, # counts philly values 
                      {"$project":{"_id":0,
                                  "Count":"$count"}},
                      {"$sort":{"Count":-1}}])
print(list(cheesesteak))

[{u'Count': 1281}]


##### Here I found all the cafe's in Philadelphia : I'm looking into how many cafes are in Philly to fill my interest to see if Philadelphia is a coffee drinking kind of city. I would like to take these findings to discuss in my concluding statement to determine if theres a way to add implementation to the OSM site to find a correlation between coffee shops and colleges. Much like what I though of earlier in my evaluation about Philly cheesesteaks being a hit amoungst college kids, and if being in a certain location in relation to colleges led to more profitability for the overall business.

In [76]:
cafe = philadelphia_pennsylvania.aggregate([ #start agregation pipeling to find cafes 
                {"$match": {"amenity": "cafe" }}, # find cafes in the data set 
                 {"$group": {"_id": '$name', "count": {"$sum": 1}}}, # group aggregation by cafe name and add to pipeline 
                 {"$sort": {"count": -1}}, # since indexing starts at 0. subract 1 to get output of 10 
                 {"$limit": 10} # gives output of 10 cafes 
])
print(list(cafe)) # prints output to notebook 

[{u'count': 45, u'_id': u'Starbucks'}, {u'count': 35, u'_id': u"Dunkin' Donuts"}, {u'count': 9, u'_id': None}, {u'count': 7, u'_id': u'Dunkin Donuts'}, {u'count': 4, u'_id': u'Green Line Cafe'}, {u'count': 3, u'_id': u'Starbucks Coffee'}, {u'count': 3, u'_id': u'OCF Coffee House'}, {u'count': 3, u'_id': u"Saxby's Coffee"}, {u'count': 2, u'_id': u'Cosi'}, {u'count': 2, u'_id': u'High Point Cafe'}]


In [77]:
# Here I found all of the colleges in Philadelphia

In [78]:
college = philadelphia_pennsylvania.aggregate([
                {"$match": {"amenity": "college" }},#find colleges 
                 {"$group": {"_id": '$name', "count": {"$sum": 1}}},# count colleges and add to list 
                 {"$sort": {"count": -1}}, # since indexing starts at 0. subtract 1 to get 10 
                 {"$limit": 10}#gives me top 10 colleges 
])
print(list(college)) # prints output to notebook 

[{u'count': 2, u'_id': u'Wilson Hall East'}, {u'count': 1, u'_id': u'Maintenance (MW)'}, {u'count': 1, u'_id': u'William K. McDaniel Integrated Learning Resource Center #401'}, {u'count': 1, u'_id': u'Liberal Arts (LA)'}, {u'count': 1, u'_id': u'Science and Allied Health (MS)'}, {u'count': 1, u'_id': u'Roosevelt Hall'}, {u'count': 1, u'_id': u'Laurel Hall'}, {u'count': 1, u'_id': u'Community College of Philadelphia - West Regional Center'}, {u'count': 1, u'_id': u'Peirce College'}, {u'count': 1, u'_id': u'Technology & Engineering Center (TEC)'}]


### Conclusion 

##### While auditing the data I updated zipcodes and addresses to a uniform format so that the data showed consistancy. I noticed while running my MongoDB aggregations that for cuisine types, the first "count" print out didn't have a category for what type of food was featured at the restaurant. So there is room for improvement with importing content into the OSM site. I would prefer if content could only be added to the site if it meets certain requirements such as having a proper cuisine description. That way when someone let's says uses an app and wants to find restaurants with a certain keyword, they can find all restaurants nearby that features that certain cuisine that they are looking for. I wouldn't see any problem implementing this feature into OSM, I feel this would be an easy fix for the data set. Also, while performing aggregations on the file using MongoDB I looked into how many colleges and cafes were in the file. With this information I would like to see if cafes located near colleges were more profitable than ones that weren't near a college. And maybe report that data to a site that may be looking for best places to build proiftable coffee shops. I definetly think there will be a correlation between cafes and universities in the data set. In order to implement the feature to find coffee and college locations you would need to use geospatial indexing. This feature would be implemented in MongoDB, and was covered in the lesson videos. Some problems you may run into would be, lets say, trying to find colleges, and Universities in the area and not find all the schools. For instance, kids in middle school in my opinion should not be drinking highly caffeinated beverages such as coffee at such a young age. So you would want to make sure the data that you are looking at is cleaned and would give the correct correlation output between coffee shops and colleges or universities( and entirely disclude k-12 schools) to present to individuals who want to open coffee shops in the area.